<a href="https://colab.research.google.com/github/paperbenni/chatbot/blob/master/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!eval 'apt install -y telnet && curl https://raw.githubusercontent.com/paperbenni/tmate-docker/master/launch.sh | bash' &> /dev/null


In [12]:
!cd ~ && cat web.txt && echo "" && cat ssh.txt

https://tmate.io/t/Y7KSbFAbhH8FGFhK39vTSTfet

ssh Y7KSbFAbhH8FGFhK39vTSTfet@ny2.tmate.io


In [0]:
!pip install -q textgenrnn
from google.colab import files
from textgenrnn import textgenrnn
from datetime import datetime
import os

In [0]:
model_cfg = {
    'word_level': False,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 3,   # number of LSTM layers (>2 recommended)
    'rnn_bidirectional': False,   # consider text both forwards and backward, can give a training boost
    'max_length': 30,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 10000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': False,   # set to True if each text has its own line in the source file
    'num_epochs': 20,   # set higher to train the model for longer
    'gen_epochs': 5,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.0,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': False   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In [0]:
!cd /content && curl "termbin.com/v1cz" > text.txt  

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17008  100 17008    0     0  60742      0 --:--:-- --:--:-- --:--:-- 60526


In [0]:
file_name = "text.txt"
model_name = 'colaboratory'   # change to set file name of resulting trained models/texts

In [0]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

Training new model w/ 3-layer, 128-cell LSTMs
Training on 13,441 character sequences.
Epoch 1/20
13/13 [==============================] - 40s 3s/step - loss: 4.4255
Epoch 2/20
13/13 [==============================] - 37s 3s/step - loss: 3.2537
Epoch 3/20
13/13 [==============================] - 38s 3s/step - loss: 3.2401
Epoch 4/20
13/13 [==============================] - 38s 3s/step - loss: 3.2321
Epoch 5/20
13/13 [==============================] - 38s 3s/step - loss: 3.2275
####################
Temperature: 0.2
####################
 ee         e       e                     e              a     e                        et               a        e         e           s t                      e                       e      t                   ha       e        e                                 o   e             t                 

 e         e      e e            e   t e      e     e   i               e           h     a  i e  te    t        e                e    t                      

In [0]:
temperature = [1.0, 0.5, 0.2, 0.2]   
prefix = None   # if you want each generated text to start with a given seed text

if train_cfg['line_delimited']:
  n = 1000
  max_gen_length = 60 if model_cfg['word_level'] else 300
else:
  n = 1
  max_gen_length = 2000 if model_cfg['word_level'] else 10000
  
timestring = datetime.now().strftime('%Y%m%d_%H%M%S')
gen_file = '{}_gentext_{}.txt'.format(model_name, timestring)

textgen.generate_to_file(gen_file,
                         temperature=temperature,
                         prefix=prefix,
                         n=n,
                         max_gen_length=max_gen_length)
files.download(gen_file)

In [0]:
files.download('{}_weights.hdf5'.format(model_name))
files.download('{}_vocab.json'.format(model_name))
files.download('{}_config.json'.format(model_name))